# Forrestania: Gravity & Magnetics (Pure Python)

```{figure} ./images/landing.png
---
scale: 30%
align: right
---
```

This case study focuses on the standalone and joint inversion of airborne magnetic data and synthetic gravity data. The magnetic data were downloaded from the [Geoscience Australia GADDS Portal](https://portal.ga.gov.au/persona/gadds), while the gravity data were generated via forward modelling specifically for this exercise based on a derived iso-shell from magnetic inversion results. We cover the following steps:

- [Data imports and pre-processing](forrestania-data)
- [Standalone inversion of gravity data](forrestania-gravity)
- [Standalone inversion of magnetic data](forrestania-magnetics)
- [Joint cross-gradient inversion](forrestania-joint)
- [K-means clustering analysis](forrestania-kmeans)

In [ ]:
# Import necessary Python libraries for data handling and visualisation
import os
import zipfile
from pathlib import Path
from tempfile import mkdtemp

import discretize
import matplotlib.pyplot as plt
import numpy as np
import pandas
import scipy as sp

# Import SimPEG library
import simpeg
from geoh5py import Workspace, objects
from PIL import Image

# Mira Geoscience specific libraries
from simpeg_drivers import assets_path

## Geological setting

The Forrestania Greenstone Belt, located in the Youanmi Terrane of the Eastern Yilgarn Craton, hosts significant Ni-Cu-PGE mineralisation, primarily within komatiitic units (Frost, 2003; Collins et al., 2012). The belt is structurally complex, featuring synclines, faults, and shear zones that play a key role in sulphide localisation. It is bounded by granitic and gneissic basement, intruded by monzogranite and granodiorite, and transected by Proterozoic dolerite dykes (Perring et al., 1995). Multiple deformation events have remobilised nickel sulphides from their original host rocks into footwall sediments and granitic intrusions, resulting in both typical and atypical mineralisation styles, including granite-hosted sulphides (Collins et al., 2012).

The study area is underlain mainly by granitic rocks, forming the basement to the Parker-Range–Hatters Hill greenstone belt immediately to the east. This N–S-trending belt is the main regional host of nickel sulphide deposits, which occur within narrow ultramafic units in an Archaean sequence of metabasalts and sulphidic sediments. Notable nearby deposits include Beautiful Sunday, Flying Fox, and New Morning.

A prominent high-magnetic anomaly, trending E–W to ENE–WSW, lies immediately west of the greenstone belt. The anomaly’s orientation, strength, and coincident elevated nickel-in-soil geochemistry suggest it may represent a mafic intrusion—making it a key target for the current geophysical inversion.

(forrestania-data-os)=
## Data Preparation

This step involves importing and formatting the necessary datasets for inversion using Mira Geoscience's libraries. For your own inversion projects, you may streamline the data preparation process by using standard Python libraries, depending on your specific needs.

### Download and unzip the dataset

We first need to unzip the package and import the data into a usable format.

The zipped package contains the following three files:
 - Airborne magnetic survey: `60472_AOI4.csv`
 - Ground gravity survey: `Forrestania_Gravity_Station_trim_.csv`
 - Digital Elevation Model (DEM): `Forrestania_SRTM1 Australia_MGA50.tiff`

In [ ]:
# Create a temporary directory for extracting the zip contents
temp_dir = Path(mkdtemp())

# Use the `assets_path()` from Mira Geoscience's simpeg_drivers to download and locate the Forrestania dataset
# This gives us the full path to the zip file we want to extract
file = assets_path() / r"Case studies/Forrestania_SRTM1 Australia_MGA50_CSV.zip"

# Print the resolved path to confirm where the file is located on disk
print(f"Dataset path: {file}")

# Extract all contents of the zip file into the temporary directory
with zipfile.ZipFile(file, "r") as zf:
    zf.extractall(temp_dir)

# List all the files that were extracted
files = list(temp_dir.iterdir())

### Load the CSV and geotiff files
We will use the `pandas` library to read the CSV files and Mira Geoscience's `geoh5py` library to read the geotiff file. 

In [ ]:
grav_survey = pandas.read_csv(next(file for file in files if "Gravity" in file.name))

### Processing Elevation Data

#### Step 1: Convert the geoImage to a 2D Grid with values

We will utilize the functionality made available in the `geoh5py` library to read and convert the geotiff to a gridded DEM with geographic coordinates.

In [1]:
# Load the elevation GeoTIFF using geoh5py
ws = Workspace()
geotiff = objects.GeoImage.create(
    ws, image=str(next(file for file in files if "SRTM1" in file.name))
)

# Register the geospatial reference system from the TIFF metadata using geoh5py
geotiff.georeferencing_from_tiff()

# Convert the image into a grid object (2D array of elevations) in geoh5py
grid = geotiff.to_grid2d()
elevations = grid.children[0].values  # Extract elevation values

# Visualise the raw elevation data (some values may be small placeholders, zeros or NaNs)
fig, ax = plt.subplots(figsize=(10, 6))
elev_image = elevations.reshape(grid.shape, order="F").T
im = ax.imshow(elev_image, cmap="terrain", origin="lower")

cbar = plt.colorbar(im, orientation="horizontal", pad=0.1)
cbar.set_label("Raw Elevation (m)")

ax.set_title("Raw Elevation Grid (including placeholders or NaNs)")
ax.set_xlabel("Grid X")
ax.set_ylabel("Grid Y")
plt.tight_layout()
plt.show()

NameError: name 'Workspace' is not defined

#### Clean Up Invalid Elevation Values
We also need to remove the zeros from the rotated DEM.

In [ ]:
# Check minimum elevation value
print(f"Minimum elevation value: {np.nanmin(elevations)}")

In [ ]:
# Some elevation values are tiny placeholders for "no data", replace with NaN
logic = np.abs(elevations) < 2e-38
elevations = np.where(logic, np.nan, elevations)

# Plot cleaned-up elevation map
fig, ax = plt.subplots(figsize=(10, 6))
cleaned_elev_image = elevations.reshape(grid.shape, order="F").T
im = ax.imshow(cleaned_elev_image, cmap="terrain", origin="lower")

cbar = plt.colorbar(im, orientation="horizontal", pad=0.1)
cbar.set_label("Cleaned Elevation (m)")

ax.set_title("Cleaned Elevation Grid (NaNs filtered)")
ax.set_xlabel("Grid X")
ax.set_ylabel("Grid Y")
plt.tight_layout()
plt.show()

In [ ]:
# Our final DEM DEM array: [X, Y, Elevation]
dem = np.c_[grid.centroids[:, :2], elevations]

### Magnetic Data Processing

#### Step 1: Extract the magnetic data and format

In [ ]:
# Load magnetic and gravity CSVs with pandas
mag_dataframe = pandas.read_csv(next(file for file in files if "AOI4" in file.name))

# Preview the dataset
print("Magnetic survey data preview:")
mag_dataframe

#### Step 2: Attach elevation to the airborne magnetic data

There are a lot more information that we need from the original CSV, but also some that are missing.
Absolute elevations for this survey are currently not available, only the height above the DEM. So we will need to extract it from the grid.
The fastest interpolation is a nearest neighbour.

In [ ]:
# Extract relevant columns: projected easting (X), northing (Y), and radar altimeter (height above terrain)
mag_locs = mag_dataframe[["X_MGA50", "Y_MGA50", "RADALT"]].to_numpy()

# Build a KDTree for fast spatial search on the DEM 2D coordinates
tree = sp.spatial.cKDTree(dem[:, :2])  # Only XY from DEM
_, ind = tree.query(mag_locs[:, :2])  # Nearest neighbour indices from DEM

# Add DEM ground elevation to RADALT to get true elevation
mag_locs[:, 2] += dem[ind, 2]

# Check elevation range
print(f"Maximum interpolated elevation: {np.nanmax(mag_locs[:, 2])}")

In [ ]:
# Plot the survey points coloured by interpolated elevation
fig, ax = plt.subplots(figsize=(10, 6))
sc = ax.scatter(
    mag_locs[:, 0],
    mag_locs[:, 1],  # X and Y coordinates
    c=mag_locs[:, 2],  # Color is based on elevation
    marker="o",  # Circle markers
    cmap="terrain",
    s=4,  # Size of points slightly larger for visibility
    edgecolors="none",
)

cbar = plt.colorbar(sc, orientation="horizontal", pad=0.1)
cbar.set_label("Interpolated Elevation (m)")

ax.set_aspect("equal")
ax.set_title("Magnetic Flight Path with Interpolated Elevation")
ax.set_xlabel("Easting (MGA50)")
ax.set_ylabel("Northing (MGA50)")
plt.tight_layout()
plt.show()

#### Step 3: Compute residual data

The source of the magnetic signal can generally be attributed to the presence or destruction of magnetic minerals in rocks (mainly magnetite). Magnetometers measure the Total Magnetic Intensity (TMI), which includes both the primary (source) field and secondary fields (signal) from the local geology. 

The inversion routine requires Residual Magnetic Intensity (RMI). The first step is to compute and remove the primary field (IGRF) at the time and location of acquisition.  The survey was conducted between February and March 1988.

##### Lookup/remove IGRF 

Information about the inducing field strength at the time and location of the survey can be accessed from the [NOAA site](https://www.ngdc.noaa.gov/geomag/calculators/magcalc.shtml?useFullSite=true#igrfwmm).

The **inducing field parameters** calculated at the time and location of the survey are

```{figure} ./images/NOAA_IGRF.png
---
scale: 20%
---
[Click to enlarge]
```

- Magnitude: 59294 nT
- Inclination: -67.1$\degree$
- Declination: -0.89$\degree$  

In [ ]:
# Reduce the data by the inducing field strength
mag_data = mag_dataframe["MAGCOMP"] - 59294
fig, ax = plt.subplots()
im = plt.hist(mag_data, bins=100)
ax.set_aspect(1)
print(f"Median: {mag_data.median()} nT")

#### Step 4: Detrend
The local background field appears to be slightly lower (~283 nT) than the computed IGRF model, as most of the data away from the main anomaly are below 0. To avoid modelling this background trend, we can remove the median value. 

In [ ]:
mag_data -= mag_data.median()

fig, ax = plt.subplots()
im = plt.hist(mag_data, bins=100)
ax.set_aspect(1)
print(f"Median: {mag_data.median()} nT")

#### Step 5: Downsampling

We can downsample the data along lines to reduce the computation cost of the inversion. Since the average flight height of this survey was 60 m, we can confidently downsample the lines by the same amount without affecting the wavelengths contained in the data.

We can do this fairly efficiently by sampling the data over a regular grid.

In [ ]:
x_grid, y_grid = np.meshgrid(
    np.arange(mag_locs[:, 0].min(), mag_locs[:, 0].max(), 60),
    np.arange(mag_locs[:, 1].min(), mag_locs[:, 1].max(), 60),
)

# Generate a KDTree from the survey
tree = sp.spatial.cKDTree(mag_locs[:, :2])

# Find the nearest indices for each grid points
_, ind = tree.query(np.c_[x_grid.flatten(), y_grid.flatten()])

mag_survey = np.c_[mag_locs, mag_data][ind, :]
print(mag_survey.shape)

In [ ]:
# Plot the survey points coloured by interpolated elevation
fig, ax = plt.subplots(figsize=(10, 6))
sc = ax.scatter(
    mag_survey[:, 0],
    mag_survey[:, 1],  # X and Y coordinates
    c=mag_survey[:, -1],  # Color is based on elevation
    marker="o",  # Circle markers
    cmap="rainbow",
    s=10,  # Size of points slightly larger for visibility
    edgecolors="none",
)

cbar = plt.colorbar(sc, orientation="horizontal", pad=0.1)
cbar.set_label("Magnetic data (nT)")

ax.set_aspect("equal")
ax.set_title("Residual (detrended) data downsampled at 60 m")
ax.set_xlabel("Easting (MGA50)")
ax.set_ylabel("Northing (MGA50)")
plt.tight_layout()
plt.show()

(forrestania-gravity-os)=
## Gravity data processing

The last dataset to format is the gravity survey. In this case, the survey is already located at the surface, and the data is already provided as terrain corrected (2.67 g/cc) in mGal. No extra transformation is required. 

In [ ]:
# Read the two CSVs with pandas
grav_dataframe = pandas.read_csv(next(file for file in files if "Gravity" in file.name))
grav_dataframe

In [ ]:
grav_survey = grav_dataframe.to_numpy()


fig, ax = plt.subplots(figsize=(10, 6))
sc = ax.scatter(
    grav_survey[:, 0],
    grav_survey[:, 1],  # X and Y coordinates
    c=grav_survey[:, -1],  # Color is based on elevation
    marker="o",  # Circle markers
    cmap="RdBu_r",
    s=10,  # Size of points slightly larger for visibility
    edgecolors="none",
)

cbar = plt.colorbar(sc, orientation="horizontal", pad=0.1)
cbar.set_label("Gravity data (mGal)")

ax.set_aspect("equal")
ax.set_title("Terrain corrected (2.67 g/cc) gravity data")
ax.set_xlabel("Easting (MGA50)")
ax.set_ylabel("Northing (MGA50)")
plt.tight_layout()
plt.show()

## Inversion

### Gravity

We begin our analysis with the ground gravity survey. We will attempt to estimate the shape and depth of the causative body at depth.

#### Create a mesh 

To be continued

(forrestania-magnetics-os)=
### Magnetics

We follow up with the processing of the airborne magnetic data.


(forrestania-joint-os)=
### Joint cross-gradient


This table summarizes the different units based on their mean density and susceptibility values. 

```{table} Summary of expected physical properties
| Unit | Susceptibility | Density |
| :--- | :--- | :---- |
| background |  low    | low      |
| A |  high    | high      |
| B |  low    | high      |
| C |  high | low |
```

## Summary

- We have inverted publicly available data over the Forrestania geological province.

- The joint cross-gradient method was successfull in promoting similarities between the different physical properties wherever possible.

- Our results suggest the presence of different geological units with distinct density and magnetic susceptibility signatures.
